## Reddit/Quora scraping

#### Requirements:

In [1]:
# pip install praw

In [2]:
import praw #Reddit scraper
import re #Regular expressions
import random #To select a comment

#### Reddit API use your own credentials:

In [3]:
reddit = praw.Reddit(
    client_id="LSZOduypQfuIUw",
    client_secret="4-RBX3XtMd52Q3FYRuuXb3I-9IhatQ",
    password="H4tutspa44word",
    user_agent="Tutorial",
    username="Hktuts",
)

Version 7.2.0 of praw is outdated. Version 7.3.0 was released 1 day ago.


#### Check if we're set up correctly (should return our reddit username)

In [4]:
print(reddit.user.me())

Hktuts


#### URL to the thread we want to scrape comments from:

In [5]:
url = "https://www.reddit.com/r/worldnews/comments/mo0vvc/norway_prime_minister_fined_1715_for_breaking/"
submission = reddit.submission(url=url)

#### Scraping reddit comments:

In [6]:
comments = []

submission.comments.replace_more(limit=None)

for comment in submission.comments.list():
    comments.append(comment.body)

print("Number of comments scraped from thread: " + str(len(comments)))

Number of comments scraped from thread: 1989


#### Picking a random comment from the first 50:

In [152]:
print(comments[(random.randint(1, 50))])

Imagine holding your head of government accountable for their actions. 

Not in America, that’s for sure.


### Quora scraping

In [213]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

In [214]:
df = pd.DataFrame({'Question': [], 'No. of answers': [],'answers':[]})

#### Scraping the question and number of answers:

In [215]:
    question_link = 'https://www.quora.com/What-is-it-like-to-live-in-Norway'
    
    #selenium settings
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument(" --window-size=1920x1080")
    
    #open instance
    driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')
    driver.get(question_link)
    time.sleep(1)
    elem = driver.find_element_by_tag_name("body")
    
    #scrape question Name
    qname = driver.find_elements_by_xpath("/html/body/div[2]/div[2]/div/div/div/div[4]/div/div/div[1]/div[1]/div/div/span/span/div/div/div/span/span")
    for value in qname:
        print("Scraped question: " + value.text)
    
    #Scrape answer count
    count = driver.find_elements_by_xpath("/html/body/div[2]/div[2]/div/div/div/div[4]/div/div/div[1]/div[2]/div[2]/div/div/div/div/div")
    for e in count:
        print(e.text + " in total")

<ipython-input-215-1709b9e1a16c>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')


Scraped question: What is it like to live in Norway?
23 Answers in total


#### Scraping answers from a question (work in progress)

In [216]:

    driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')
    
    # give the url to scrap
    driver.get(question_link)
        
    # define pause time for browser
    SCROLL_PAUSE_TIME = 1

	# get browser source
    html_source = driver.page_source
    question_count_soup = BeautifulSoup(html_source, 'html.parser')

	# Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    answer_set = set()

    #infinite while loop, break it when you reach the end of the page or not able to scroll further.
    while True:
            html_source = " "
            i = 0

	#try to scroll 20 times in case of slow connection
            while i < 20:

                   #Scroll down to one page length
                   driver.execute_script("window.scrollBy(0, 1080);")

				#  Wait to load page
                   time.sleep(SCROLL_PAUSE_TIME)

				#  get page height in pixels
                   new_height = driver.execute_script("return document.body.scrollHeight")

				#  break this loop when you are able to scroll futher
                   if new_height != last_height:
                        
                        break     
                        
                   i += 1
                   
            # get html page source
            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            #question_link is the class for questions
            answer_texts = soup.find_all('span', attrs={'class' : 'q-box qu-userSelect--text'})
            

		    # add questions to a set for uniqueness
            for answer in answer_texts:
                
                answer = str(answer).lstrip("<div class=\"ui_qtext_expanded\"><span class=\"ui_qtext_rendered_qtext\">").rstrip("</span></div>")
                answer_set.add(answer)

            # not able to scroll further, break the infinite loop
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

            print('Total Answers: ' + str(len(answer_set)))
            break
            
            #quit browser finally when our scraping is done            
    browser.quit()


<ipython-input-216-2bf8bb0e99be>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=r'chromedriver.exe')


Total Answers: 52


In [196]:
print(soup.prettify())

<html class="js-focus-visible" data-js-focus-visible="" dir="ltr" lang="en" style="padding: 0px; margin: 0px; --vh:10.8px;">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="https://qsf.fs.quoracdn.net/-4-images.favicon.ico-26-ebf6a9e7f7b4576d.ico" rel="icon"/>
  <title>
   What is it like to live in Norway? - Quora
  </title>
  <meta content="noarchive" name="robots"/>
  <meta content="light" name="twitter:widgets:theme"/>
  <link href="https://www.quora.com/What-is-it-like-to-live-in-Norway" rel="canonical"/>
  <link as="script" href="https://qsbr.fs.quoracdn.net/-4-ans_frontend-relay-common-27-83be92f679a7a4fa.webpack" nonce="" rel="preload"/>
  <link as="script" href="https://qsbr.fs.quoracdn.net/-4-ans_frontend-relay-component-Modals-NUX-27-00214488ed291386.webpack" nonce="" rel="preload"/>
  <link as="script" href="https://qsbr.fs.quoracdn.net/-4-ans_frontend-relay-page-QuestionPageLoadable-27-65b2e15bc0d94885.webpack" nonce="" rel="pre